In [499]:
#libraries
import pandas as pd
import re
from datetime import datetime
from dateutil import parser

In [500]:
#list of all changes needed
# removing 2 last columns
# populating species
# changing date to a date type instead of string
# populating country
# area
# case number


In [501]:
# importing data file
data = pd.read_csv('GSAF5.csv', encoding='latin-1')


In [503]:
#removing 2 columns duplicated
data.drop(columns=['Case Number.1', 'Case Number.2'], inplace=True)
#removing 2 last columns that do not bring relevant information
data.drop(columns=['Unnamed: 22', 'Unnamed: 23'], inplace=True)


In [506]:
#cleaning year column

#when there is value in Date in this format CA number BC and year = 0
for ind in data[['Date','Year']][(data['Date'].str.upper().str.startswith('CA') ) & (data['Year'] == 0)].index:  
    data['Year'][ind] = "".join(re.findall(r'\b\d+\b',data['Date'][ind] ))

############### cleaning Date Column ###############

# if starts with before, between or no date will be set as No Date
#data['Date'][(data['Year'] == 0)& ( data['Date'].str.upper().str.startswith( ('BEF','BET','NO DATE'))) ] = 'No Date'

#rules defined
#   1. will save all trash information for further analysis
#   2. will take all records that contains a valid date as real one, unless it will be checked with business
#   3. year could be valid until checking data column
#   4. year and month could be valid






C:\Users\David\AppData\Local\Temp/ipykernel_1624/30240033.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Year'][ind] = "".join(re.findall(r'\b\d+\b',data['Date'][ind] ))


In [508]:
#getting bag of words
bag_of_words = []
# bag_of_words.append
for ind in data.index:
    try:
        #applying an upper and replace and strip in case that there is an strange character at the begining or by the end
        base = data['Date'][ind].upper().replace('--','-').strip(' `.')
        #valildating that it could be a correct date
        parser.parse(data['Date'][ind])
        #in case that is a date, will replace a space or dot to - 
        data['Date'][ind] = base.replace(' ','-').replace('.','-')
    except:
        #if there is no valid date data clenaning process starts saving all garbage information in bag_of_words
        #spliting base by space
        for val in base.split(' '):
            #validating if it is a date
            if re.match(r'^\d{1,2}-[A-Z]+-\d{2,4}$',val):
                pass
            else:
                #if it is not a date, is it a digit?
                if val.isdigit():
                    pass
                else:
                    #validating specific cases
                    if val == '190FEB-2010':
                        data['Date'][ind] = '19-FEB-2010'
                        val = '19-FEB-2010'
                    elif val in ['1844.07.16.R','1836.07.26.R']:
                        data['Date'][ind] = val.replace('.R','').replace('.','-')
                        val = val.replace('.R','').replace('.','-')
                    elif val == '23-DECP1896':
                        data['Date'][ind] = '23-DEC-1896'
                        val = '23-DEC-1896'
                    elif val == '26-SEP-T937':
                        data['Date'][ind] = '26-SEP-1937'
                        val = '26-SEP-1937'
                    elif val.endswith(('.B','.A')):
                        data['Date'][ind] = val.replace('.B','').replace('.A','')
                        val = val.replace('.B','').replace('.A','')
                    elif val == '1836.00':
                        bag_of_words.append(val)
                    try:
                        #if this validation proceed check again if it is a valid date
                        parser.parse(val)
                    except:
                        #if it is not, add that value to bag of words
                        bag_of_words.append(val)

C:\Users\David\AppData\Local\Temp/ipykernel_1624/3733795762.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][ind] = base.replace(' ','-').replace('.','-')
C:\Users\David\AppData\Local\Temp/ipykernel_1624/3733795762.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][ind] = '19-FEB-2010'
C:\Users\David\AppData\Local\Temp/ipykernel_1624/3733795762.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['D

In [511]:
for ind in data.index:
    #removing characters that are not allowed
    val = data['Date'][ind].upper().replace('--','-').strip(' `.-')
    new_string = ''
    for dat in val.split(' '):
        if dat in bag_of_words:
            new_string = new_string+''
        else:
            new_string = new_string + dat
    try:
        #check length if it is a valid one
        if len(new_string)>3:
            #checking if last part of string is a valid date
            parser.parse(new_string)
            # over writing our new string 
            data['Date'][ind] = new_string.strip()
        else:
            data['Date'][ind] = 'No Date'    
    except:
        data['Date'][ind] = 'No Date'
    
        

C:\Users\David\AppData\Local\Temp/ipykernel_1624/2679173459.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][ind] = new_string.strip()
C:\Users\David\AppData\Local\Temp/ipykernel_1624/2679173459.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][ind] = 'No Date'
C:\Users\David\AppData\Local\Temp/ipykernel_1624/2679173459.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'][ind] = 'No Date'


In [512]:
#will complement Date year value with the year value
# if year value is 0 then will take year from date

In [513]:
################################## Cleaning Year ##################################
# removing year from Date column in case Year column has value

for ind in data.index:
    if data['Date'][ind] != 'No Date':
        if data['Year'][ind] > 0:
            # adding values of year 4 digits
            data['Date'].iloc[ind] = re.sub(r'-\d{2}$|-\d{4}$','-'+str(data['Year'][ind]),data['Date'][ind]) 
        # if year is 0 and date has a year set value of Date to Year column
        elif data['Year'][ind] == 0 and len(data['Date'][ind]) == 4:
            data['Year'].iloc[ind] = data['Date'][ind]
            data['Date'].iloc[ind] = 'No Date'
        # removing days from date column
        data['Date'].iloc[ind] = re.sub(r'^\d{1,2}-','',data['Date'][ind]) 
    elif len(data['Date'][ind]) < 5:
        data['Date'].iloc[ind] = 'No Date'


#data.head(30)

C:\Users\David\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [514]:
#removing values with Date = 0 or Year = 'No Date'
data.drop(data[(data['Year'] == 0) & (data['Date'] == 'No Date')].index, inplace=True)



In [519]:
################################## Checking Country ##################################

#Populating null values with unknown value
data['Country'] = data['Country'].fillna('Unknown', inplace=True)
#Passing information about ocean or sea in name to Area column
for country_sea in data['Country'][(~data['Country'].isnull() ) & (data['Country'].str.endswith((' SEA',' OCEAN')) )].index:
    data['Area'].iloc[country_sea] = data['Country'].iloc[country_sea]
    data['Country'].iloc[country_sea] = 'Unknown'

#have previously check id with Between word in country name
#data['Country'][(~data['Country'].isnull() ) & (data['Country'].str.find('&') != -1) & (data['Country'].str.find('Between') != -1) ] 
# = data['Country'][5861]
data['Country'][(~data['Country'].isnull() ) & (data['Country'].str.find('&') != -1) & (data['Country'].str.find('Between') != -1) ].replace('Between','')

#removing all countries with ? by the end of the string
data['Country'][(~data['Country'].isnull() ) & (data['Country'].str.find('?') != -1) ] = 'Unknown' 


C:\Users\David\AppData\Local\Temp/ipykernel_1624/3270166267.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Country'][(~data['Country'].isnull() ) & (data['Country'].str.find('?') != -1) ] = 'Unknown'


In [520]:
################################## Checking sex ##################################

def checking_name(row):
    val = ''
    if str(row['Name']).upper().strip() == 'MALE' or str(row['Name']).upper().strip() == 'BOY':
        return 'M'
    elif str(row['Name']).upper().strip() == 'FEMALE' or str(row['Name']).upper().strip() == 'GIRL':
        return 'F'
    return

bag_of_words_names = []
#Checking all values in name
#there are male and female, girl and boy
for ind in data['Name'].index:
    for i in str(data['Name'][ind]).split(' '):
        bag_of_words_names.append(i)


print(data.columns)
#Column has an space at the end
data.rename(columns={'Sex ':'Sex'}, inplace=True)
#Creating another column that contains sex values in name column
data['Sex_in_name'] = data.apply(checking_name, axis=1)
data['Sex'].fillna(data['Sex_in_name'], inplace = True)

#Checking values of sex
print(data['Sex'].unique())
#removing space in one M sex
data['Sex'] = data['Sex'].str.strip()
data['Sex'] = data['Sex'].fillna('unknown')
data['Sex'] = data['Sex'].str.replace('lli','unknown')
data['Sex'] = data['Sex'].str.replace('.','unknown')
data['Sex'] = data['Sex'].str.replace('N','unknown')
print(data['Sex'].unique())




Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'original order'],
      dtype='object')
['M' None 'F' 'M ' 'lli' 'N' '.']
['M' 'unknown' 'F']


C:\Users\David\AppData\Local\Temp/ipykernel_1624/4239715494.py:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Sex'] = data['Sex'].str.replace('.','unknown')


In [521]:
################################## Checking Fatal(Y/N) ##################################

#Checking unique values of this column
data['Fatal (Y/N)'].unique()
#['N', 'Y', nan, 'UNKNOWN', ' N', 'F', 'N ', '#VALUE!', 'n']

data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.upper().str.strip()
data['Fatal (Y/N)'].unique()

data['Fatal (Y/N)'] = data['Fatal (Y/N)'].fillna('UNKNOWN')
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.replace('#VALUE!','UNKNOWN')
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.replace('F','Y')




In [522]:
################################## Checking Types ##################################
# Checking types
data['Type'].unique()

data['Type'] = data['Type'].str.replace('Boating','Boat')
data['Type'] = data['Type'].str.replace('Invalid','Unknown')



In [525]:
################################## Species ##################################
data.rename(columns={'Species ':'Species'}, inplace=True)
data.columns
#cleaning all na values
data['Species'] = data['Species'].fillna('UNKNOWN', inplace=True)
#have extract all possible kind of sharks
types_of_sharks = []
types_of_sharks_cleaned = []

def shark_species(row):
    valid_sharks = ''
    for shark in re.findall(r'[A-Za-z]{3,} [S|s]hark', row['Species']):
        if shark.upper() in types_of_sharks_cleaned:
            if valid_sharks != '':
                valid_sharks = valid_sharks+'/'+shark.upper()
            else:
                valid_sharks = shark.upper()
    if valid_sharks == '':
        return row
    else:
        return valid_sharks


for ind in data.index:
    types_of_sharks.append(re.findall(r'[A-Za-z]{3,} [S|s]hark', str(data['Species'][ind])))

types_of_sharks_cleaned = [shark.upper() for list_sharks in types_of_sharks for shark in list_sharks]
#Checking unique sharks
types_of_sharks_cleaned = list(np.unique(types_of_sharks_cleaned))
#removing the ones which are clearly wrong
types_of_sharks_cleaned.remove('OTHER SHARK')
types_of_sharks_cleaned.remove('ANOTHER SHARK')
types_of_sharks_cleaned.remove('COLORED SHARK')
types_of_sharks_cleaned.remove('DOG SHARK')
types_of_sharks_cleaned.remove('UNIDENTIFIED SHARK')
types_of_sharks_cleaned.remove('FOR SHARK')
types_of_sharks_cleaned.remove('FOOT SHARK')
types_of_sharks_cleaned.remove('FROM SHARK')
types_of_sharks_cleaned.remove('FEMALE SHARK')
types_of_sharks_cleaned.remove('LARGER SHARK')
types_of_sharks_cleaned.remove('LARGE SHARK')
types_of_sharks_cleaned.remove('CAPTURED SHARK')
types_of_sharks_cleaned.remove('CAPTIVE SHARK')
types_of_sharks_cleaned.remove('COCKTAIL SHARK')
types_of_sharks_cleaned.remove('LITTLE SHARK')
types_of_sharks_cleaned.remove('METRE SHARK')
types_of_sharks_cleaned.remove('TWO SHARK')
types_of_sharks_cleaned.remove('YOUNG SHARK')
types_of_sharks_cleaned.remove('SMALL SHARK')
types_of_sharks_cleaned.remove('BELIEVED SHARK')
types_of_sharks_cleaned.remove('JUVENILE SHARK')
types_of_sharks_cleaned.remove('RED SHARK')
types_of_sharks_cleaned.remove('THE SHARK')
types_of_sharks_cleaned.remove('SAME SHARK')
types_of_sharks_cleaned.remove('SAW SHARK')

print(np.unique(types_of_sharks_cleaned))

data['Species'] = data.apply( shark_species, axis = 1)


ValueError: list.remove(x): x not in list

In [530]:
################################## removing new column sex_in_name ##################################
#data.drop('Sex_in_name', axis='columns', inplace=True)
data.columns
################################## exporting file as csv ##################################
data.to_csv('Sharks_cleaned.csv')